In [1]:
import sys 
sys.path.append("../")
from wave_circuit import * 
from mps_utils import * 
from density import * 

from qiskit.quantum_info import Statevector

In [2]:
sigma = 0.1 
mu = 0.5
n = 10
N = 2**n

f = lambda x: 2/(np.sqrt(3 * sigma) * (np.pi)**(1/4)) * (1 - ((x-mu)/sigma)**2 ) * np.exp(- (x-mu)**2/(2*sigma**2))
df = lambda x: -4/(np.sqrt(3 * sigma) * (np.pi)**(1/4)) * (x-mu)/(sigma**2) * np.exp(- (x-mu)**2/(2*sigma**2)) - (x-mu)/(sigma**2) * f(x)


init_state = f(np.linspace(0,1-1/N,N) + 1/(2*N))
init_state = np.concatenate([init_state, np.zeros(N)])
K = np.linalg.norm(init_state)
init_state = init_state/K

f_extend = lambda x : f(x - np.floor(x))/K
df_extend = lambda x : df(x - np.floor(x))/K


f_evo = lambda x,t :  1/2 * (f_extend(x - t) + f_extend(x + t))
dxf_evo = lambda x,t :1/2 * (df_extend(x - t) + df_extend(x + t))
dtf_evo = lambda x,t : 1/2 * (-df_extend(x - t) + df_extend(x + t))

In [3]:
opt_layers = np.load("../data/mps_opt_data.npy", allow_pickle=True)[0]['opt_layers']
exact_state = f(np.linspace(0,1-1/N,N) + 1/(2*N))
exact_state = exact_state/np.linalg.norm(exact_state)

In [4]:
Fid = []
R = []
Fid_pure = []
R_pure = []

for idx in range(len(opt_layers)):
    state_prep_circ = QuantumCircuit(n)
    state_prep_circ = state_prep_circ.compose(convert_layers_to_circuit(opt_layers[idx]), range(1,n))
    state_prep_circ.h(0)
    for i in range(1,n):
        state_prep_circ.cx(0, i)
    
    state_prep_circ = full_reduce(state_prep_circ)
    f1 = []
    f_pure = [] 
    r = []
    r_pure = []

    for lam in 10**(np.linspace(-5, -3, 21)):
        rho = get_density_matrix(state_prep_circ,lam)
        rho_pure=(rho @ rho)/np.trace(rho @ rho)
        
        f1.append(1-abs(exact_state.conjugate().T @ rho @ exact_state))
        r.append(rho)
        f_pure.append(1-abs(exact_state.conjugate().T @ rho_pure @ exact_state))
        r_pure.append(rho_pure)
        
    Fid.append(f1)
    Fid_pure.append(f_pure)
    R.append(r)
    R_pure.append(r_pure)
    print('----------------------------')

----------------------------
----------------------------
----------------------------
----------------------------
----------------------------
----------------------------


In [5]:
np.save("../data/purified.npy",[{
    'Fid':Fid, 
    'Fid_pure':Fid_pure,
    'R':R,
    'R_pure':R_pure
}], allow_pickle=True)

In [6]:
def n_int(fun):
    return 1/N * np.sum([fun(_) for _ in np.linspace(0,1-1/N,N) + 1/(2*N)])

def T(t):
    return n_int(lambda x : dtf_evo(x,t)**2)

def U(t):
    return n_int(lambda x : dxf_evo(x,t)**2)

In [7]:
eval_times = np.linspace(0,1,51)
lam  = 10**np.linspace(-5,-4,10)
n_layers = len(opt_layers)
swap = np.kron(np.eye(2), swap_matrix(n))
U_op =-N * swap @ np.block([[np.diag(spectrum_P(N)), np.zeros((N,N))], [np.zeros((N,N)), np.zeros((N,N))]]) @ swap
T_op =-N * swap @ np.block([[np.zeros((N,N)), np.zeros((N,N))], [np.zeros((N,N)), np.diag(spectrum_P(N))]]) @ swap


state_prep_circ = QuantumCircuit(n)
state_prep_circ = state_prep_circ.compose(convert_layers_to_circuit(opt_layers[idx]), range(1,n))
state_prep_circ.h(0)
for i in range(1,n):
    state_prep_circ.cx(0, i)

rho_list = []
full_circuit = QuantumCircuit(n+1)
full_circuit = full_circuit.compose(state_prep_circ, range(1,n+1))
full_circuit = full_circuit.compose(energy_circuit(n, swap=False), range(n+1))

for l in lam:
    R_l = []
    for t in eval_times:
        circ = full_circuit.assign_parameters([t])
        circ = full_reduce(circ)
        rho = get_density_matrix(circ, l)
        R_l.append(rho)
    rho_list.append(R_l)
    print("----------------------------------------------------")

----------------------------------------------------
----------------------------------------------------
----------------------------------------------------
----------------------------------------------------
----------------------------------------------------
----------------------------------------------------
----------------------------------------------------
----------------------------------------------------
----------------------------------------------------
----------------------------------------------------


In [8]:
data = []
for i, (rho_list_i, lam_i) in enumerate(zip(rho_list[::-1], lam[::-1])):
    
    rho_pure = [(rho @ rho) / np.trace(rho @ rho) for rho in rho_list_i]

    U_vals = np.real([np.trace(U_op @ r) for r in rho_pure])
    T_vals = np.real([np.trace(T_op @ r) for r in rho_pure])

    noisy_U_vals = np.real([np.trace(U_op @ r) for r in rho_list_i])
    noisy_T_vals = np.real([np.trace(T_op @ r) for r in rho_list_i])
    data.append([U_vals, T_vals, noisy_U_vals, noisy_T_vals])
    
U_exact = np.array([U(t) for t in eval_times])
T_exact = np.array([T(t) for t in eval_times])


In [9]:
np.save("../data/purified_sim.npy",
        [{
        'data':data,
        'U_exact':U_exact, 
        'T_exact':T_exact,
        'times':eval_times,
        'lam':lam,
        }], allow_pickle=True)